In [1]:
from keras.models import Sequential, save_model, load_model
from keras.layers import Dense, Dropout
from keras import optimizers, regularizers
from datetime import datetime
from keras.callbacks import ModelCheckpoint, TensorBoard
import numpy as np
from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
random_state = 6
np.random.seed(random_state)

Using TensorFlow backend.


### Define parameters

In [2]:
no_of_layers = 2
learning_rate = 0.001
training_epochs = 50
batch_size = 128
keep_prob = 1.0

In [22]:
from keras.utils import to_categorical
filename = "seismogram_data.csv"
df = pd.read_csv(filename)
df.describe()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,177,178,179,180,181,182,183,184,185,186
count,1606.000000,1606.000000,1606.000000,1606.000000,1606.000000,1606.000000,1606.000000,1606.000000,1606.000000,1606.000000,...,1.606000e+03,1.606000e+03,1.606000e+03,1606.0,1606.000000,1606.000000,1.606000e+03,1.606000e+03,1606.000000,1606.000000
mean,451.221669,941.665095,75.003741,9.231527,50.089629,9.011473,33.163889,11.137501,21.313839,13.738046,...,1.792764e+07,1.693866e+07,1.685018e+07,0.0,278.243072,-0.188070,3.743952e+12,5.021586e+17,31.972242,0.323163
std,302.649662,225.061154,37.099307,35.261332,19.673425,23.210687,12.591131,14.331587,7.673028,8.545568,...,1.728873e+08,1.703474e+08,1.699784e+08,0.0,7662.201633,4.893058,2.781032e+13,4.202786e+18,763.247083,0.467830
min,0.000000,487.638894,0.590916,-95.650386,9.991572,-63.162621,5.273298,-38.585637,0.495500,-20.883890,...,2.773652e+03,2.870772e+03,2.719687e+03,0.0,-18424.601562,-148.536407,1.145826e+00,1.647420e+08,-1.557898,0.000000
25%,200.250000,775.150225,46.703585,-14.839313,36.362204,-7.899532,24.523875,2.047651,16.663696,9.773171,...,4.122496e+03,3.969333e+03,3.962479e+03,0.0,-0.564912,-0.066578,1.089125e+05,4.726152e+10,-0.056514,0.000000
50%,401.000000,900.926968,74.069958,12.228716,49.900223,12.684427,31.645115,14.229327,20.592639,15.489321,...,1.058606e+04,1.002841e+04,9.615704e+03,0.0,0.114271,-0.004160,1.769261e+06,7.682032e+11,0.241702,0.000000
75%,684.750000,1095.790912,100.281783,35.329806,63.719153,28.028384,40.849423,22.416433,25.514643,19.247300,...,7.547047e+04,7.190346e+04,6.694036e+04,0.0,2.218386,0.057529,6.593582e+08,8.217131e+14,1.950677,1.000000
max,1086.000000,1572.073401,202.898203,98.485401,123.096246,64.457973,81.809122,47.306912,52.774850,39.409499,...,3.862672e+09,3.835163e+09,3.825628e+09,0.0,300014.312500,11.173301,3.098055e+14,6.909585e+19,27426.243439,1.000000


In [46]:
X = df.iloc[:,:186]
Y = df.iloc[:,-1]
X, Y = shuffle(X, Y, random_state=random_state)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=random_state)
# Y_train = to_categorical(Y_train)
# Y_test = to_categorical(Y_test)

mms = StandardScaler()
X_train = mms.fit_transform(X_train)
X_test = mms.fit_transform(X_test)

input_layer = np.shape(X_train)[1]
output_layer = 1

### Model setup

In [54]:
model = Sequential()
model.add(Dense(units = 128, activation = 'relu', input_dim = input_layer, name='layer_1'))
model.add(Dense(units = 64, activation = 'relu', name='layer_2'))
model.add(Dense(units = 32, activation = 'relu', name='layer_3'))
model.add(Dense(units = 16, activation = 'relu', name='layer_4'))
model.add(Dense(units = output_layer, activation="softmax", name='Output_layer'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_1 (Dense)              (None, 128)               23936     
_________________________________________________________________
layer_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
layer_3 (Dense)              (None, 32)                2080      
_________________________________________________________________
layer_4 (Dense)              (None, 16)                528       
_________________________________________________________________
Output_layer (Dense)         (None, 1)                 17        
Total params: 34,817
Trainable params: 34,817
Non-trainable params: 0
_________________________________________________________________


### Training

In [55]:
history = model.fit(X_train, Y_train, batch_size = 10, epochs = training_epochs, validation_split=0.0, verbose=0)

In [56]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
Y_pred = model.predict(X_test)

print('The accuracy of the model is {}'.format(accuracy_score(Y_test, Y_pred)))
print('# Classification report \n {}'.format(classification_report(Y_test, Y_pred)))

The accuracy of the model is 0.3132780082987552
# Classification report 
              precision    recall  f1-score   support

        0.0       0.00      0.00      0.00       331
        1.0       0.31      1.00      0.48       151

avg / total       0.10      0.31      0.15       482



/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Plot loss and accuracy 

In [57]:
#print(history.history.keys())

plt.figure(figsize=(20,5))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss', size=16)
plt.xlabel('epoch', size=16)
plt.legend(['train', 'test'], loc='upper right')
plt.grid()
plt.show()